In [2]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import urllib.request, urllib.parse
import time
from datetime import datetime


## CNN


In [753]:
headlines = []
dates = []
bodies = []
urls = []

In [754]:
pagenum = 1
url = f'https://search.api.cnn.io/content?q=contact%20tracing&page={pagenum}&sort=newest&from={(pagenum-1)*10}'
json_data = requests.get(url).json()
data = json_data['result']



In [755]:
for pagenum in range(1,51): #do 51
    url = f'https://search.api.cnn.io/content?q=contact%20tracing&page={pagenum}&sort=newest&from={(pagenum-1)*10}'
#     print(url)
    json_data = requests.get(url).json()
    data = json_data['result']

    for d in data:
        headlines.append(d['headline'])
        dates.append(d['firstPublishDate'])
        bodies.append(d['body'])
        urls.append(d['url'])


https://search.api.cnn.io/content?q=contact%20tracing&page=1&sort=newest&from=0
https://search.api.cnn.io/content?q=contact%20tracing&page=2&sort=newest&from=10
https://search.api.cnn.io/content?q=contact%20tracing&page=3&sort=newest&from=20
https://search.api.cnn.io/content?q=contact%20tracing&page=4&sort=newest&from=30
https://search.api.cnn.io/content?q=contact%20tracing&page=5&sort=newest&from=40
https://search.api.cnn.io/content?q=contact%20tracing&page=6&sort=newest&from=50
https://search.api.cnn.io/content?q=contact%20tracing&page=7&sort=newest&from=60
https://search.api.cnn.io/content?q=contact%20tracing&page=8&sort=newest&from=70
https://search.api.cnn.io/content?q=contact%20tracing&page=9&sort=newest&from=80
https://search.api.cnn.io/content?q=contact%20tracing&page=10&sort=newest&from=90
https://search.api.cnn.io/content?q=contact%20tracing&page=11&sort=newest&from=100
https://search.api.cnn.io/content?q=contact%20tracing&page=12&sort=newest&from=110
https://search.api.cnn.i

In [759]:
cnn = pd.DataFrame.transpose(pd.DataFrame([headlines, dates, urls, bodies]))
cnn.columns = ['title','date', 'url', 'text']


In [761]:
cnn.to_csv('cnn.csv')

## Fox

In [256]:
titles = []
links = []

In [257]:
daterange = [(20200101, 20200301), (20200301, 20200401), (20200401, 20200415), 
             (20200415, 20200501), (20200501, 20200510), (20200510, 2020520),
             (20200520, 20200601), (20200601, 20200610), (20200610, 20200620)]

In [258]:
for date in daterange:
    for page in range(0,10): 
        startdate, enddate = date
        print(startdate, enddate)
        url = f"https://api.foxnews.com/search/web?q=contact%20tracing+more:pagemap:metatags-pagetype:article+more:pagemap:metatags-dc.type:Text.Article&siteSearch=foxnews.com&siteSearchFilter=i&sort=date:r:{startdate}:{enddate}&start={page*10}&callback=__jp{page+1}"


        req = requests.get(url)
        t = re.findall(r'\s"og:title":\s"(.*)"',req.text)
        urls = re.findall(r'\s"link":\s"(.*)"',req.text)
        titles.extend(t)
        links.extend(urls)
        print(page, len(t), len(urls))
        time.sleep(5)


20200101 20200301
0 10 10
20200101 20200301
1 10 10
20200101 20200301
2 10 10
20200101 20200301
3 10 10
20200101 20200301
4 10 10
20200101 20200301
5 0 0
20200101 20200301
6 0 0
20200101 20200301
7 0 0
20200101 20200301
8 0 0
20200101 20200301
9 0 0
20200301 20200401
0 10 10
20200301 20200401
1 10 10
20200301 20200401
2 10 10
20200301 20200401
3 10 10
20200301 20200401
4 10 10
20200301 20200401
5 10 10
20200301 20200401
6 7 7
20200301 20200401
7 0 0
20200301 20200401
8 0 0
20200301 20200401
9 0 0
20200401 20200415
0 10 10
20200401 20200415
1 10 10
20200401 20200415
2 10 10
20200401 20200415
3 10 10
20200401 20200415
4 10 10
20200401 20200415
5 3 3
20200401 20200415
6 0 0
20200401 20200415
7 0 0
20200401 20200415
8 0 0
20200401 20200415
9 0 0
20200415 20200501
0 10 10
20200415 20200501
1 10 10
20200415 20200501
2 10 10
20200415 20200501
3 10 10
20200415 20200501
4 10 10
20200415 20200501
5 10 10
20200415 20200501
6 10 10
20200415 20200501
7 10 10
20200415 20200501
8 10 10
20200415 20200

In [259]:
len(titles), len(links)

(446, 446)

In [260]:
bodies = []
dates = []

In [261]:
index = 1
for url in links:
#     print(index)
    index += 1
#     url = "https://www.foxnews.com/health/johns-hopkins-releases-online-contact-tracing-course"
    req = urllib.request.Request(url)

    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')
    
    if not ("radio" in url or "video" in url or "watch/" in url or "c/brooke-crothers" in url 
            or "shows/ingraham" in url or "w-abby-hornacek" in url) :
        date = soup.find("div", attrs = {"class": "article-date"}).text
        date = re.match(r'Published\s(.*)', date).group(1)
        dates.append(date)
        a = soup.find("div", attrs = {"class": "article-body"})
        text = [b.text for b in a.findAll("p")]
        bodies.append(text)
    else:
        bodies.append("NA")
        dates.append("NA")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [262]:
len(titles), len(links), len(dates), len(bodies)

(446, 446, 446, 446)

In [263]:
fox = pd.DataFrame.transpose(pd.DataFrame([titles, dates, links, bodies]))
fox.columns = ['title','date', 'url', 'text']

In [264]:
fox.date = fox.date.astype(str) + ' 2020'


In [265]:
# https://stackoverflow.com/questions/47246143/changing-relative-times-to-actual-dates-in-a-pandas-dataframe

absolute_date = pd.to_datetime(fox.date, errors="coerce")
relative_date = (datetime.today() - 
                 fox.date.str.extract("(.*) ago", expand=False).apply(pd.Timedelta))

date = relative_date.combine_first(absolute_date)


In [266]:
fox.date = date.dt.date

In [267]:
fox.to_csv("fox.csv")

## NYT

In [155]:
titles = []
dates = []
urls = []

apikey = "I1EX29cHWoOBYDsre22OfzeTkxYujx0B"


In [156]:
for page in range(100):
#     print(page)
    url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=contact%20tracing&sort=best&begin_date=20200101&end_date=20200613&page={page}&api-key={apikey}'
    time.sleep(5)

    req = requests.get(url)

    for j in req.json()['response']['docs']:
        titles.append(j['headline']['main'])
        dates.append(j['pub_date'])
        urls.append(j['web_url'])
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [158]:
nyt = pd.DataFrame.transpose(pd.DataFrame([titles, dates, urls]))
nyt.columns = ['title', 'date','url']
nyt

,title,date,url
0,Contact Tracing Is Harder Than It Sounds,2020-05-23T15:00:06+0000,https://www.nytimes.com/2020/05/23/sunday-revi...
1,"As the Nation Begins Virus Tracing, It Could L...",2020-05-21T09:00:12+0000,https://www.nytimes.com/2020/05/21/nyregion/co...
2,5 Rules to Live By During a Pandemic,2020-06-09T09:00:13+0000,https://www.nytimes.com/2020/06/09/well/live/c...
3,Giving Your Number to Strangers? It’s Not Flir...,2020-06-07T09:55:54+0000,https://www.nytimes.com/2020/06/07/world/europ...
4,"Testing Is Key to Beating Coronavirus, Right? ...",2020-05-29T16:16:58+0000,https://www.nytimes.com/2020/05/29/world/asia/...
...,...,...,...
995,How Dorothea Lange Defined the Role of the Mod...,2020-02-10T21:33:53+0000,https://www.nytimes.com/2020/02/10/t-magazine/...
996,Many Facebook Employees to Work From Home Perm...,2020-05-21T04:08:04+0000,https://www.nytimes.com/2020/05/21/business/co...
997,U.S. to Suspend Most Travel From Europe; N.B.A...,2020-03-11T04:04:56+0000,https://www.nytimes.com/2020/03/11/world/coron...
998,"No Touch, No Hands-On Learning, for Now, as Mu...",2020-05-29T09:00:37+0000,https://www.nytimes.com/2020/05/29/arts/design...


In [166]:
for index, row in nyt.iterrows():
    print(index, row.url)
    req = urllib.request.Request(row.url)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')

    # date = soup.find("time")["datetime"]
    a = soup.find("section", attrs={"name": "articleBody"})
    if a:
        body = ' '.join([c.text for c in a.findAll("p")][:-3])
        nyt.loc[index, 'body'] = body
    
    time.sleep(5)

0 https://www.nytimes.com/2020/05/23/sunday-review/coronavirus-contact-tracing.html
1 https://www.nytimes.com/2020/05/21/nyregion/contact-tracing-paterson-nj.html
2 https://www.nytimes.com/2020/06/09/well/live/coronavirus-rules-pandemic-infection-prevention.html
3 https://www.nytimes.com/2020/06/07/world/europe/germany-coronavirus-reopening.html
4 https://www.nytimes.com/2020/05/29/world/asia/japan-coronavirus.html
5 https://www.nytimes.com/2020/05/24/business/the-week-in-business-coronavirus-work-office-jobless.html
6 https://www.nytimes.com/2020/05/18/health/coronavirus-contact-tracing-jobs.html
7 https://www.nytimes.com/2020/05/20/world/europe/virus-paris-reopening.html
8 https://www.nytimes.com/2020/05/14/nyregion/coronavirus-de-blasio-mitchell-katz.html
9 https://www.nytimes.com/2020/05/14/opinion/coronavirus-communication.html
10 https://www.nytimes.com/2020/06/03/health/coronavirus-contact-tracing-apps.html
11 https://www.nytimes.com/reuters/2020/06/04/technology/04reuters-healt

91 https://www.nytimes.com/2020/03/19/world/europe/europe-china-coronavirus.html
92 https://www.nytimes.com/reuters/2020/06/11/us/11reuters-health-coronavirus-usa-mnuchin.html
93 https://www.nytimes.com/2020/06/04/us/politics/coronavirus-infection-demographics.html
94 https://www.nytimes.com/2020/05/12/world/europe/germany-coronavirus-r-number.html
95 https://www.nytimes.com/2020/04/30/opinion/coronavirus-warm-weather-mutation.html
96 https://www.nytimes.com/2020/05/11/health/coronavirus-second-wave-infections.html
97 https://www.nytimes.com/2020/03/02/us/coronavirus-schools-rhode-island.html
98 https://www.nytimes.com/2020/04/29/world/europe/coronavirus-france-masks.html
99 https://www.nytimes.com/interactive/2020/04/24/opinion/coronavirus-covid-19-georgia-reopen.html
100 https://www.nytimes.com/reuters/2020/06/03/sports/rugby/03reuters-health-coronavirus-sport-newzealand.html
101 https://www.nytimes.com/aponline/2020/06/05/business/ap-virus-outbreak-japan-mystery-of-success.html
102 

186 https://www.nytimes.com/2020/04/13/us/politics/peter-navarro-coronavirus.html
187 https://www.nytimes.com/2020/05/19/nyregion/coronavirus-ny-nyc-live-news.html
188 https://www.nytimes.com/2020/05/20/nyregion/coronavirus-nyc.html
189 https://www.nytimes.com/2020/04/14/insider/reader-questions-coronavirus.html
190 https://www.nytimes.com/2020/05/31/nyregion/coronavirus-new-york-live-updates.html
191 https://www.nytimes.com/2020/06/08/world/australia/new-zealand-coronavirus-ardern.html
192 https://www.nytimes.com/2020/05/16/nyregion/coronavirus-new-york-update.html
193 https://www.nytimes.com/2020/05/04/opinion/coronavirus-global-cooperation-cities.html
194 https://www.nytimes.com/2020/03/13/opinion/coronavirus-best-response.html
195 https://www.nytimes.com/interactive/2020/06/08/upshot/when-epidemiologists-will-do-everyday-things-coronavirus.html
196 https://www.nytimes.com/2020/05/04/opinion/coronavirus-sweden-herd-immunity.html
197 https://www.nytimes.com/2020/05/11/us/coronavirus-

286 https://www.nytimes.com/2020/05/12/world/coronavirus-global-live-news.html
287 https://www.nytimes.com/2020/05/11/nyregion/coronavirus-new-york-update.html
288 https://www.nytimes.com/2020/05/18/world/middleeast/iran-coronavirus-surge.html
289 https://www.nytimes.com/2020/05/12/nyregion/coronavirus-nyc.html
290 https://www.nytimes.com/2020/05/11/us/politics/coronavirus-hearing-fauci-quarantine.html
291 https://www.nytimes.com/2020/05/24/health/coronavirus-face-shields.html
292 https://www.nytimes.com/2020/02/28/health/coronavirus-pence-messaging.html
293 https://www.nytimes.com/2020/05/19/arts/dance/dance-streaming.html
294 https://www.nytimes.com/2020/05/25/health/coronavirus-testing-trump.html
295 https://www.nytimes.com/2020/03/05/briefing/coronavirus-japan-stock-markets-blued-app.html
296 https://www.nytimes.com/2020/05/15/world/coronavirus-women-leaders.html
297 https://www.nytimes.com/2020/04/10/world/asia/coronavirus-migrants.html
298 https://www.nytimes.com/2020/04/06/smart

386 https://www.nytimes.com/article/who-is-melissa-derosa.html
387 https://www.nytimes.com/2020/04/21/world/coronavirus-cases-global.html
388 https://www.nytimes.com/2020/04/28/opinion/coronavirus-testing.html
389 https://www.nytimes.com/2020/05/18/us/coronavirus-updates.html
390 https://www.nytimes.com/2020/04/23/nyregion/coronavirus-new-york-update.html
391 https://www.nytimes.com/2020/05/11/world/coronavirus-news.html
392 https://www.nytimes.com/2020/05/14/opinion/coronavirus-reopen-workers.html
393 https://www.nytimes.com/2020/05/22/nyregion/coronavirus-ny-nj-live-cases-deaths.html
394 https://www.nytimes.com/2020/05/26/us/coronavirus-live-updates.html
395 https://www.nytimes.com/2020/04/17/world/coronavirus-news-updates.html
396 https://www.nytimes.com/2020/05/04/world/coronavirus-world-cases-deaths.html
397 https://www.nytimes.com/2020/05/20/us/coronavirus-cases-deaths.html
398 https://www.nytimes.com/aponline/2020/06/12/business/ap-virus-outbreak-the-latest.html
399 https://www.

...

583 https://www.nytimes.com/aponline/2020/06/04/sports/soccer/ap-soc-china-hao.html
584 https://www.nytimes.com/2020/04/04/parenting/coronavirus-kids-safe-online.html
585 https://www.nytimes.com/2020/01/30/world/europe/italy-coronavirus-cruise.html
586 https://www.nytimes.com/2020/02/28/sports/soccer/silvio-berlusconi-monza.html
587 https://www.nytimes.com/2020/05/18/business/wework-rent-coronavirus.html
588 https://www.nytimes.com/2020/02/03/us/coronavirus-united-states-cases.html
589 https://www.nytimes.com/2020/05/27/sports/chess-coronavirus-irina-krush.html
590 https://www.nytimes.com/2020/03/19/reader-center/coronavirus-donald-mcneil.html
591 https://www.nytimes.com/2020/03/18/us/virus-census-homeless.html
592 https://www.nytimes.com/2020/03/10/us/politics/biden-sanders-coronavirus.html
593 https://www.nytimes.com/2020/05/11/style/shopping-will-shopping-ever-be-the-same.html
594 https://www.nytimes.com/aponline/2020/06/11/us/ap-us-america-protests-senator-race-comments.html
595 ht

681 https://www.nytimes.com/2020/01/27/briefing/wuhan-coronavirus-kobe-bryant-john-bolton.html
682 https://www.nytimes.com/2020/03/16/business/coronavirus-bills-charity.html
683 https://www.nytimes.com/2020/01/20/well/family/bilingual-toddlers-china-learning-english.html
684 https://www.nytimes.com/2020/04/03/opinion/coronavirus-farm-workers.html
685 https://www.nytimes.com/aponline/2020/06/05/sports/baseball/ap-bbo-virus-outbreak-mlb-proposal-analysis.html
686 https://www.nytimes.com/aponline/2020/06/01/sports/baseball/ap-bbo-virus-outbreak-baseball.html
687 https://www.nytimes.com/2020/05/12/obituaries/sister-georgianna-glose-dead-coronavirus.html
688 https://www.nytimes.com/2020/04/10/us/coronavirus-san-francisco-homeless-shelter.html
689 https://www.nytimes.com/2020/02/20/sports/football/nfl-cba-meaning.html
690 https://www.nytimes.com/2020/01/24/style/typography-font-design.html
691 https://www.nytimes.com/2020/04/02/nyregion/coronavirus-new-york-update.html
692 https://www.nytime

776 https://www.nytimes.com/2020/01/27/learning/lesson-plans/teach-the-super-bowl-ideas-for-subjects-across-the-curriculum.html
777 https://www.nytimes.com/2020/03/04/business/united-airlines-coronavirus.html
778 https://www.nytimes.com/2020/03/26/opinion/sunday/doctors-coronavirus-safety.html
779 https://www.nytimes.com/2020/03/04/arts/dance/riverdance-25th-anniversary.html
780 https://www.nytimes.com/2020/03/18/us/coronavirus-nebraska-biocontainment.html
781 https://www.nytimes.com/2020/03/19/opinion/coronavirus-pregnancy.html
782 https://www.nytimes.com/2020/02/04/arts/television/gene-reynolds-dead.html
783 https://www.nytimes.com/2020/05/20/opinion/art-physics-brian-greene.html
784 https://www.nytimes.com/2020/04/10/nyregion/coronavirus-help-healthcare-workers.html
785 https://www.nytimes.com/2020/03/18/us/coronavirus-zoos-webcams.html
786 https://www.nytimes.com/2020/05/09/nyregion/coronavirus-new-york-update.html
787 https://www.nytimes.com/2020/03/31/briefing/coronavirus-masks-w

872 https://www.nytimes.com/2020/01/23/world/australia/plane-crash-fires.html
873 https://www.nytimes.com/2020/03/30/business/coronavirus-retail-furloughs-macys.html
874 https://www.nytimes.com/2020/03/20/parenting/grandparents-visit-safety-coronavirus.html
875 https://www.nytimes.com/2020/03/16/business/media/cuomo-new-york-coronavirus.html
876 https://www.nytimes.com/2020/03/09/opinion/coronavirus-south-korea-church.html
877 https://www.nytimes.com/2020/01/28/books/review/friendship-lydia-denworth.html
878 https://www.nytimes.com/2020/03/18/opinion/nyc-poor-coronavirus.html
879 https://www.nytimes.com/2020/06/10/us/politics/national-guard-protests.html
880 https://www.nytimes.com/interactive/2020/02/18/magazine/google-revolt.html
881 https://www.nytimes.com/2020/05/30/world/americas/virus-central-america-dengue.html
882 https://www.nytimes.com/2020/03/30/business/japan-economy-coronavirus.html
883 https://www.nytimes.com/2020/05/15/us/virus-piano-lessons.html
884 https://www.nytimes.

969 https://www.nytimes.com/2020/06/12/us/george-floyd-white-protesters.html
970 https://www.nytimes.com/2020/06/10/arts/dance/alvin-ailey-retirements-coronavirus.html
971 https://www.nytimes.com/2020/04/08/education/college-ai-chatbots-students.html
972 https://www.nytimes.com/2020/03/05/learning/teenagers-in-the-times-february-2020.html
973 https://www.nytimes.com/2020/02/27/learning/how-concerned-are-you-about-the-coronavirus-outbreak.html
974 https://www.nytimes.com/2020/01/15/automobiles/nissan-carlos-ghosn-strategy.html
975 https://www.nytimes.com/2020/03/24/business/coronavirus-per-country-pandemic.html
976 https://www.nytimes.com/2020/03/05/arts/design/nyc-this-weekend-art-and-museums.html
977 https://www.nytimes.com/2020/05/28/world/asia/indonesia-coronavirus-surge.html
978 https://www.nytimes.com/2020/06/01/fashion/weddings/how-to-be-your-own-glam-squad-for-a-zoom-wedding.html
979 https://www.nytimes.com/2020/05/04/briefing/vaccine-china-censorship-markets.html
980 https://ww

In [167]:
nyt.to_csv("nyt.csv")

## USA Today

In [3]:
url = "https://www.usatoday.com/search/?q=contact+tracing"
req = urllib.request.Request(url)

html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'lxml')

In [4]:
titles = []
urls = []
dates = []
bodies = []

In [764]:
for page in range (1,6):
    url = f"https://www.usatoday.com/search/?q=contact%20tracing&page={page}"
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')

    a = soup.find("div", attrs={"class": "gnt_pr"})
    b = a.findAll("a")
    for c in b:
        if ("/search/?q" not in c["href"] and "/sitemap" not in c["href"]):
            titles.append(c.text)
            urls.append("https://www.usatoday.com" + c["href"])
            dates.append(c.find("div")["data-c-dt"])

In [765]:
len(titles), len(urls), len(dates)

(100, 100, 100)

In [410]:
for url in urls:
#     print(url)
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')

    # date = soup.find("time")["datetime"]
    a = soup.findAll("p", attrs={"class":"gnt_ar_b_p"})
    
    if not a:
        
    
    bodies.append(' '.join([b.text for b in a]))

https://www.usatoday.com/story/travel/cruises/2020/06/01/coronavirus-cruise-ships-far-uvc-contact-tracing-could-future/5199946002/
https://www.usatoday.com/story/news/factcheck/2020/05/26/fact-check-bill-gates-did-not-help-craft-hr-6666-contact-tracing-bill/5220823002/
https://www.usatoday.com/story/opinion/voices/2020/06/03/coronavirus-contact-tracer-covid-19-california-column/5309779002/
https://www.usatoday.com/story/news/factcheck/2020/05/28/fact-check-trace-act-hr-6666-wont-forcibly-remove-people-their-homes/5199873002/
https://www.usatoday.com/story/tech/2020/05/22/covid-19-contact-tracing-app-care-19-privacy-policy/5247445002/
https://www.usatoday.com/story/news/factcheck/2020/05/27/fact-check-trace-act-hr-6666-does-not-permit-expanded-surveillance/5228333002/
https://www.usatoday.com/story/news/factcheck/2020/05/27/fact-check-trace-acts-funding-tied-prohibiting-unvaccinated/5219417002/
https://www.usatoday.com/story/tech/2020/05/20/apple-and-google-release-coronavirus-contact-t

https://www.usatoday.com/story/news/50-states/2020/06/07/theme-parks-speedy-clocks-rent-relief-news-around-states/111918816/
https://www.usatoday.com/story/entertainment/music/2020/05/01/coronavirus-when-concerts-return-expect-much-different-experience/3055806001/
https://www.usatoday.com/story/news/investigations/2020/05/31/george-floyd-protest-agitators-mostly-homegrown-not-outsiders/5300362002/
https://www.usatoday.com/story/tech/2020/05/26/apple-stores-reopen-but-differently-you-may-not-able-enter/5260800002/
https://www.usatoday.com/story/tech/2020/05/18/coronavirus-tracking-tech-issues/5217928002/
https://www.usatoday.com/story/news/50-states/2020/06/09/infected-ferry-crater-lake-mask-shaming-virtual-farmers-market-news-around-states/111939810/
https://www.usatoday.com/story/tech/columnist/2020/06/04/george-floyd-police-protests-tech-facial-recognition-coronavirus/3149030001/
https://www.usatoday.com/story/opinion/2020/05/27/coronavirus-covid-19-testing-symptoms-column/5264406002

In [411]:
len(bodies)

100

In [412]:
usatoday = pd.DataFrame.transpose(pd.DataFrame([titles, dates, urls, bodies]))
usatoday.columns = ['title', 'date','url', 'text']
usatoday.head()

,title,date,url,text
0,Tech cruise ships could implement to combat sp...,"Jun. 3, 2020",https://www.usatoday.com/story/travel/cruises/...,"Today, the world faces an unmatched challenge:..."
1,Fact check: Bill Gates did not craft contact t...,"May 27, 2020",https://www.usatoday.com/story/news/factcheck/...,As communities around the country begin to eas...
2,I'm a coronavirus contact tracer. Asking vulne...,"Jun. 3, 2020",https://www.usatoday.com/story/opinion/voices/...,"Just three months ago, I worked as a librarian..."
3,Fact Check: Contact tracing won't result in pe...,"May 28, 2020",https://www.usatoday.com/story/news/factcheck/...,As states begin to reopen and Americans return...
4,Tech privacy firm warns coronavirus contact tr...,"May 22, 2020",https://www.usatoday.com/story/tech/2020/05/22...,"SIOUX FALLS, S.D. – A contact tracing app push..."
...,...,...,...,...
95,Trump says US will pass 10 million coronavirus...,"May 12, 2020",https://www.usatoday.com/story/news/politics/2...,WASHINGTON — President Donald Trump said Monda...
96,NCAA medical chief explains what it might take...,"May 16, 2020",https://www.usatoday.com/story/sports/ncaaf/20...,Two weeks from the date some college athletics...
97,"Socially-distant concerts from Keith Urban, Tr...","May 20, 2020",https://www.usatoday.com/story/entertainment/m...,"NASHVILLE, Tenn. — The last time bassist Jon J..."
98,NCAA's Mark Emmert: Scale of college sports ma...,"May 15, 2020",https://www.usatoday.com/story/sports/college/...,NCAA President Mark Emmert said Friday that th...


In [785]:
# One randomly special case

url = usatoday.loc[9].url
req = urllib.request.Request(url)
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'lxml')

a = soup.find("section", attrs={"class":"in-depth-content"})
text = ' '.join([b.text for b in a.findAll("p")])

In [783]:
usatoday.loc[9, "text"] = text

In [784]:
usatoday.to_csv("usatoday.csv")

## Huffington Post

In [468]:
titles = []
urls = []
dates = []

In [493]:
url = "https://search.huffpost.com/search;_ylt=Awr9CW2gKeZe9IcATSJsBmVH;_ylu=X3oDMTEza3NiY3RnBGNvbG8DZ3ExBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=1&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=Awr9DuCjKeZe_PwAA1dsBmVH;_ylu=X3oDMTEza3NiY3RnBGNvbG8DZ3ExBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=11&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=Awr9CWnDKeZeR0wAMyNsBmVH;_ylu=X3oDMTEza3NiY3RnBGNvbG8DZ3ExBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=21&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=Awr9Dt3TKeZewHAAb1BsBmVH;_ylu=X3oDMTEza3NiY3RnBGNvbG8DZ3ExBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=31&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=Awr9Dt_jKeZezsgAh1dsBmVH;_ylu=X3oDMTEza3NiY3RnBGNvbG8DZ3ExBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=41&pz=10&bct=0&xargs=0"
# url = "https://search.huffpost.com/search;_ylt=A2KLfSK7K.Ze1bAAqCRsBmVH;_ylu=X3oDMTEzajVvczlrBGNvbG8DYmYxBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=51&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=AwrJ7J_WK.ZeKMkAIFJsBmVH;_ylu=X3oDMTEzajVvczlrBGNvbG8DYmYxBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=61&pz=10&bct=0&xargs=0"
# url = "https://search.huffpost.com/search;_ylt=A0geK.DwK.ZehecA0PBsBmVH;_ylu=X3oDMTEzajVvczlrBGNvbG8DYmYxBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=71&pz=10&bct=0&xargs=0"
# url = "https://search.huffpost.com/search;_ylt=A2KLfSgDLOZeZvgAzLhsBmVH;_ylu=X3oDMTEzajVvczlrBGNvbG8DYmYxBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=81&pz=10&bct=0&xargs=0"
url = "https://search.huffpost.com/search;_ylt=A0geKYwbLOZeOycAqJpsBmVH;_ylu=X3oDMTEzajVvczlrBGNvbG8DYmYxBHBvcwMxBHZ0aWQDBHNlYwNwYWdpbmF0aW9u?p=contact+tracing&pz=10&fr=huffpost&bct=0&b=91&pz=10&bct=0&xargs=0"


req = urllib.request.Request(url)

html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'lxml')
lst = soup.findAll("li", attrs={"class": "ov-a"})
print(len(lst))
for l in lst[1:]:
    title = l.find("h4").find("a")
    titles.append(title.text)
    urls.append(title["href"])
    dates.append(l.find("span", attrs={"class": "pl-15"}).text)


0


In [492]:
len(titles), len(urls), len(dates)

(53, 53, 53)

In [494]:
titles

['A Contact Tracing Expert On The Challenges Of Preventing The Coronavirus',
 "What Is A Contact Tracer's Job Like During Coronavirus? Let Them Explain.",
 'What Are Contact Tracers, And Will They Slow The Spread Of COVID-19?',
 'The U.S. Is Building A Contact-Tracer Army',
 'Rudy Giuliani Acts As If Cancer, Obesity Were Contagious In Fox News Coronavirus Rant',
 "San Francisco Launches System To Trace All Coronavirus Patients' Contacts",
 "I Was Exposed To Coronavirus In Singapore. Here's What Their Rapid Response Looks Like.",
 'Protests, Erosion Of Trust In Government Upend Coronavirus Control',
 "'Contact Tracing' For Ebola Explained In One Infographic",
 'Can The UK Avoid A Second Coronavirus Peak When Lockdown Lifts?',
 "'S**thole' Countries Have Handled The Coronavirus Better Than The United States",
 'Provincetown, Long A Haven For Outsiders, Debates How Wide To Open Its Doors',
 'Coronavirus Live Updates: Read The Latest About The COVID-19 Outbreak',
 'Indian Govt Trots Out Me

In [495]:
huffington = pd.DataFrame.transpose(pd.DataFrame([titles, dates, urls]))
huffington.columns = ['title', 'date','url']
huffington

,title,date,url
0,A Contact Tracing Expert On The Challenges Of ...,"May 19, 2020",https://www.huffpost.com/entry/contact-tracing...
1,What Is A Contact Tracer's Job Like During Cor...,"May 14, 2020",https://www.huffpost.com/entry/contact-tracer-...
2,"What Are Contact Tracers, And Will They Slow T...","May 13, 2020",https://www.huffpost.com/entry/contract-tracer...
3,The U.S. Is Building A Contact-Tracer Army,"May 17, 2020",https://www.huffpost.com/entry/coronavirus-con...
4,"Rudy Giuliani Acts As If Cancer, Obesity Were ...","April 23, 2020",https://www.huffpost.com/entry/rudy-giuliani-f...
5,San Francisco Launches System To Trace All Cor...,"April 15, 2020",https://www.huffpost.com/entry/san-francisco-c...
6,I Was Exposed To Coronavirus In Singapore. Her...,"April 16, 2020",https://www.huffpost.com/entry/coronavirus-sin...
7,"Protests, Erosion Of Trust In Government Upend...","June 1, 2020",https://www.huffpost.com/entry/protests-upend-...
8,'Contact Tracing' For Ebola Explained In One I...,"October 1, 2014",https://www.huffingtonpost.com/entry/contact-t...
9,Can The UK Avoid A Second Coronavirus Peak Whe...,"April 24, 2020",https://www.huffpost.com/entry/can-the-uk-avoi...


In [496]:
huffington.to_csv("huffington_url.csv")

## Washington Post

There were too many to scrape all in one go so I had to do them in batches

In [35]:
urls = []


In [36]:
for start in range(0, 3000, 100):

    if start == 0: url = "https://sitesearchapp.washingtonpost.com/sitesearch-api/v2/search.json?count=500&datefilter=displaydatetime:%5B*+TO+NOW%2FDAY%2B1DAY%5D&facets.fields=%7B!ex%3Dinclude%7Dcontenttype,%7B!ex%3Dinclude%7Dname&highlight.fields=headline,body&highlight.on=true&highlight.snippets=1&query=contact+tracing&sort=&startat=1&callback=angular.callbacks._0"
    else:          url = f"https://sitesearchapp.washingtonpost.com/sitesearch-api/v2/search.json?count=500&datefilter=displaydatetime:%5BNOW%2FDAY-1YEAR+TO+NOW%2FDAY%2B1DAY%5D&facets.fields=%7B!ex%3Dinclude%7Dcontenttype,%7B!ex%3Dinclude%7Dname&highlight.fields=headline,body&highlight.on=true&highlight.snippets=1&query=contact+tracing&sort=&startat={start}&callback=angular.callbacks._0"

    req = urllib.request.Request(url)

    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')

    c = re.findall(r'https://www.washingtonpost.com(.*?)"', soup.text)
    print(start, len(c))

    urls.extend(["https://www.washingtonpost.com" + b for b in c])

0 100
100 100
200 99
300 99
400 98
500 100
600 98
700 98
800 99
900 99
1000 49
1100 0
1200 0
1300 0
1400 0
1500 0
1600 0
1700 0
1800 0
1900 0
2000 0
2100 0
2200 0
2300 0
2400 0
2500 0
2600 0
2700 0
2800 0
2900 0


In [55]:
titles = []
dates = []
bodies = []

In [57]:
for url in urls:
    # url = "https://www.washingtonpost.com/national/contact-tracing-is-best-tool-we-have-until-theres-a-vaccine-say-health-experts/2020/06/13/94f42ffa-a73b-11ea-bb20-ebf0921f3bbd_story.html"

    if "video" in url:
        title = "NA"
        date = "NA"
        body = "NA"
    else:
        try:
            req = urllib.request.Request(url)
            html = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(html, 'lxml')

            title = soup.find("h1").text

            d = soup.find("div", attrs={"class": "display-date"})
            if d:
                date = d.text
            else:
                d = soup.find("span", attrs={"class": "author-timestamp"})
                if d:
                    date = d["content"]
                else: date = "NA"

            body = ' '.join([b.text for b in soup.findAll("p")])

        except:
            body = 'NA'
            date = 'NA'
            title = 'NA'


    print(index, title, date)
    bodies.append(body)
    titles.append(title)
    dates.append(date)

    index += 1
    time.sleep(1)


https://www.washingtonpost.com/politics/the-health-202-us-isnt-ready-for-the-contact-tracing-it-needs-to-stem-the-coronavirus/2020/06/15/58dbe256-499f-4f04-8133-487d4f5530f5_story.html
The Health 202: U.S. isn't ready for the contact tracing it needs to stem the coronavirus 2020-06-15T07:24-500
https://www.washingtonpost.com/national/contact-tracing-is-best-tool-we-have-until-theres-a-vaccine-say-health-experts/2020/06/13/94f42ffa-a73b-11ea-bb20-ebf0921f3bbd_story.html
Contact tracing is ‘best’ tool we have until there’s a vaccine, health experts say June 14, 2020 at 7:10 PM EDT
https://www.washingtonpost.com/technology/2020/06/01/contact-tracing-congress-privacy/
Members of Congress to unveil bipartisan bill to regulate contact-tracing apps, fearing potential privacy abuses June 1, 2020 at 2:32 PM EDT
https://www.washingtonpost.com/business/technology/britain-scraps-virus-tracing-app-for-google-apple-system/2020/06/18/13e3ee40-b17d-11ea-98b5-279a6479a1e4_story.html
Britain scraps viru

The Daily 202: Australian app downloads underscore how far the U.S. lags on coronavirus contact tracing 2020-04-27T10:22-500
https://www.washingtonpost.com/dc-md-va/2020/05/08/virginia-reopen-coronavirus-faq/
All of Virginia moves to next phase of reopening. Here’s what that means. June 15, 2020 at 12:24 PM EDT
https://www.washingtonpost.com/politics/the-daily-202-two-conservatives-join-liberals-in-supreme-court-ruling-to-protect-lgbtq-workers/2020/06/15/b4081844-e71e-40f8-9d84-171173194214_story.html
The Daily 202: Two conservatives join liberals in Supreme Court ruling to protect LGBTQ workers 2020-06-15T12:26-500
https://www.washingtonpost.com/business/how-a-second-wave-sparks-renewed-pandemic-fears/2020/06/12/29a87444-acbe-11ea-a43b-be9f6494a87d_story.html
How a ‘Second Wave’ Sparks Renewed Pandemic Fears June 15, 2020 at 3:01 AM EDT
https://www.washingtonpost.com/politics/power-up-trump-tries-to-sweep-the-coronavirus-pandemic-under-the-rug-as-cases-surge/2020/06/15/89d6a0cb-1193-4

The Health 202: The hydroxychloroquine hype is over 2020-06-16T07:39-500
https://www.washingtonpost.com/local/dc-region-will-have-enough-testing-and-tracing-capacity-by-july-to-contain-coronavirus-officials-say/2020/05/24/6f6e59d2-9c5e-11ea-ad09-8da7ec214672_story.html
D.C. region will have enough testing and tracing capacity by July to contain coronavirus, officials say May 25, 2020 at 5:01 AM EDT
https://www.washingtonpost.com/opinions/2020/04/03/we-need-tech-government-help-with-contact-tracing-that-doesnt-have-mean-big-brother/
We need tech and government help with contact tracing. That doesn’t have to mean Big Brother. April 3, 2020 at 5:02 PM EDT
https://www.washingtonpost.com/world/asia_pacific/asia-today-more-controls-used-in-beijing-as-outbreak-grows/2020/06/15/fc99df78-af75-11ea-98b5-279a6479a1e4_story.html
Asia Today: More controls used in Beijing as outbreak grows June 15, 2020 at 10:06 PM EDT
https://www.washingtonpost.com/opinions/a-devastating-second-wave-is-possible-but

Migrant farmworkers, many coronavirus positive, move north from Florida to other states June 11, 2020 at 10:12 AM EDT
https://www.washingtonpost.com/sports/soccer/the-latest-kentucky-derby-festival-cuts-large-scale-events/2020/06/11/08204ede-ac5d-11ea-a43b-be9f6494a87d_story.html
The Latest: Houston suspending all voluntary workouts June 11, 2020 at 11:30 PM EDT
https://www.washingtonpost.com/health/2020/05/11/this-course-will-train-an-army-contact-tracers-you-can-take-it-too/
This course will train an army of contact tracers. You can take it, too. May 11, 2020 at 8:00 AM EDT
https://www.washingtonpost.com/business/ducey-touts-hospitals-amid-focus-on-rise-in-virus-cases/2020/06/11/9d8d19ec-ac40-11ea-a43b-be9f6494a87d_story.html
Ducey touts hospitals amid focus on rise in virus cases June 11, 2020 at 8:06 PM EDT
https://www.washingtonpost.com/nation/2020/06/11/2-million-coronavirus-cases-us/
U.S. eclipses 2 million coronavirus cases June 11, 2020 at 7:40 PM EDT
https://www.washingtonpos

Tracing South Korea’s latest virus outbreak shoves LGBTQ community into unwelcome spotlight May 11, 2020 at 8:05 AM EDT
https://www.washingtonpost.com/opinions/2020/05/29/case-reopening-schools-this-fall/
The case for reopening schools this fall May 29, 2020 at 12:18 PM EDT
https://www.washingtonpost.com/technology/2020/05/20/apple-google-api-launch/
Apple and Google launch coronavirus exposure software May 20, 2020 at 1:56 PM EDT
https://www.washingtonpost.com/politics/istanbul-residents-ignore-virus-rules-on-first-weekend-out/2020/06/07/26843d4a-a8e3-11ea-a43b-be9f6494a87d_story.html
Istanbul residents ignore virus rules on first weekend out June 7, 2020 at 1:20 PM EDT
https://www.washingtonpost.com/politics/the-technology-202-coronavirus-crisis-has-not-stopped-antitrust-crackdown-in-washington/2020/05/18/c62f0516-1640-45e6-b17a-a23d026531f1_story.html
The Technology 202: Coronavirus crisis has not stopped antitrust crackdown in Washington 2020-05-18T09:15-500
https://www.washingtonp

...

Region’s hospitals are now ready to handle an expected covid-19 surge — on one condition May 11, 2020 at 5:00 AM EDT
https://www.washingtonpost.com/video/postlive/wplive/mozilla-ceo-mitchell-baker-what-starts-out-as-voluntarycan-easily-become-quite-coercive/2020/05/13/612e37f0-30c6-4c29-9b2c-8f338f1220ec_video.html
NA NA
https://www.washingtonpost.com/postlive/wplive/mozilla-ceo-mitchell-baker-what-starts-out-as-voluntarycan-easily-become-quite-coercive/2020/05/13/612e37f0-30c6-4c29-9b2c-8f338f1220ec_video.html
NA NA
https://www.washingtonpost.com/video/postlive/wplive/rhode-island-governor-says-data-collected-by-new-app-would-be-automatically-destroyed/2020/05/13/1689f47b-342e-46f4-857c-452b61933115_video.html
NA NA
https://www.washingtonpost.com/postlive/wplive/rhode-island-governor-says-data-collected-by-new-app-would-be-automatically-destroyed/2020/05/13/1689f47b-342e-46f4-857c-452b61933115_video.html
NA NA
https://www.washingtonpost.com/politics/michigan-cancels-contract-with-two-

Here Are the States Where Reopening Is Riskiest May 7, 2020 at 9:01 PM EDT
https://www.washingtonpost.com/opinions/2020/05/05/so-your-state-is-opening-up-dont-let-down-your-guard/
As states reopen, here’s how you protect yourself from the coming surge May 5, 2020 at 4:04 PM EDT
https://www.washingtonpost.com/opinions/as-coronavirus-deaths-continue-the-trump-show-gets-even-more-surreal/2020/05/04/387fb6c0-8e29-11ea-a9c0-73b93422d691_story.html
The moment when Trump’s schtick finally failed May 4, 2020 at 3:46 PM EDT
https://www.washingtonpost.com/opinions/2020/05/08/larry-hogan-recovery-doesnt-mean-going-back-normal/
Larry Hogan: Recovery doesn’t mean going back to normal May 8, 2020 at 2:52 PM EDT
https://www.washingtonpost.com/health/2020/05/07/coronavirus-solutions/
 Guides to the other side  NA
https://www.washingtonpost.com/health/2020/04/14/cdc-fema-have-created-plan-reopen-america-heres-what-it-says/
CDC, FEMA have created a plan to reopen America. Here’s what it says. April 14, 

Anti-vaccination leaders seize on coronavirus to push resistance to inoculation  May 5, 2020 at 7:00 AM EDT
https://www.washingtonpost.com/politics/trump-task-forces-coronavirus-pandemic/2020/04/11/5cc5a30c-7a77-11ea-a130-df573469f094_story.html
Trump administration has many task forces — but still no plan for beating covid-19 April 11, 2020 at 9:45 PM EDT
https://www.washingtonpost.com/opinions/2020/04/12/second-most-dangerous-contagion-america-conservative-irrationality/
The second-most-dangerous contagion in America: Conservative irrationality  April 12, 2020 at 7:30 AM EDT
https://www.washingtonpost.com/opinions/the-long-war-against-the-coronavirus/2020/05/04/91abfbf2-8be3-11ea-80df-d24b35a568ae_story.html
The long war against the coronavirus May 4, 2020 at 4:26 PM EDT
https://www.washingtonpost.com/opinions/were-asking-an-extraordinary-amount-from-state-and-local-governments-they-deserve-more-aid/2020/05/04/898dfd82-8baa-11ea-9dfd-990f9dcc71fc_story.html
Congress, don’t cripple fr

The Technology 202: The coronavirus could worsen some of Amazon’s political problems 2020-04-28T08:51-500
https://www.washingtonpost.com/politics/the-cybersecurity-202-ohio-primary-marks-a-major-test-for-mail-in-voting/2020/04/28/865f3f72-0570-460e-89f9-151e8042069f_story.html


IncompleteRead: IncompleteRead(190336 bytes read)

In [59]:
len(urls), len(bodies), len(titles), len(dates)

(1039, 436, 436, 436)

In [ ]:
wp = pd.DataFrame.transpose(pd.DataFrame([titles, dates, urls, bodies]))
wp.columns = ['title', 'date','url', 'text']
wp.drop_duplicates(inplace=True)
wp.to_csv(f"washingtonpost{range_end}.csv")

### Concat

Need to merge these bc scraping took too long

In [206]:
from os import listdir
from os.path import isfile, join
wp_csvs = ["scripts/"+f for f in listdir("scripts") if ".csv" in f]

In [207]:
dfs = [pd.read_csv(a) for a in wp_csvs]

In [208]:
null_urls = df[df.text.isnull()].url.values
print(len(null_urls))

# Drop the null values
df = pd.concat(dfs)
df = df[df.date.notna()]
df = df[df.text.notna()]

1


In [5]:
df.reset_index(drop=True, inplace=True)
df.drop(columns="Unnamed: 0", inplace=True)


NameError: name 'df' is not defined

In [210]:
# Fix the dates
df['date2'] = pd.to_datetime(df.date, errors="coerce")
a = df[df['date2'].isnull()].date.str.extract("(.*)T", expand=False)
df.loc[a.index, "date2"] = a.values

/Users/yenniejun/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/yenniejun/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [211]:
df[df['date2'].isnull()]

,title,date,url,text,date2


In [212]:
df['date'] = pd.to_datetime(df['date2'], utc=True).dt.date
df.drop(columns="date2", inplace=True)

In [213]:
df[df.text.isnull()]

,title,date,url,text


In [215]:
df.to_csv("washingtonpost.csv")